<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/LogisticRegression_fwd_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tqdm
import torch.distributions as distr

In [3]:
%pip install torchmetrics --quiet

In [4]:
import torchmetrics
import torchvision
from torchvision import transforms as T

Loading and preprocessing the data

In [5]:
#Load the dataset
train_data = torchvision.datasets.KMNIST('./data', train=True, download=True)

In [6]:
#To get the first element
for image, label in train_data:
  break

In [7]:
# Simple transformation that converts the PIL image to a PyTorch array
T.ToTensor()(image).shape

torch.Size([1, 28, 28])

In [8]:
# T.Compose allows to chain together multiple transformations
train_transforms = T.Compose([
    T.ToTensor()
])

In [9]:
# This loads data with both data conversion.
train_data = torchvision.datasets.KMNIST('./data', train=True, transform=train_transforms)

In [10]:
# Loaders are used to shuffle, batch, and possibly sample the elements of the dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [11]:
xb, yb = next(iter(train_loader))
print(xb.shape)
print(yb.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [12]:
# Loading the test data is similar, but (a) we do not apply data augmentation,
# and (b) we do not shuffle when building the mini-batches.
test_data = torchvision.datasets.KMNIST('./data', train=False, transform=T.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Define Logistic Regression

In [13]:
from torch import nn
from torch.nn import functional as F

In [14]:
class SimpleLogisticRegression(nn.Module):
  def __init__(self, input_size, w, b):
    super(SimpleLogisticRegression, self).__init__()
    input_size = 28
    self.linear = nn.Linear(input_size, 28*28)
    self.linear.weight = nn.Parameter(w)
    self.linear.bias = nn.Parameter(b)


  def forward(self, x):
    x = x.reshape(-1, 28*28)
    return torch.softmax(self.linear(x), 1)

In [15]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Initialize the parameters

In [16]:
# We initialize the parameters randomly and the model with an input size
w = torch.randn((10, 28*28), requires_grad=False)
b = torch.randn(10, requires_grad=False)
LG = SimpleLogisticRegression(1, w, b)

In [17]:
# Note: we also need to move data when asking for a prediction
LG(xb).shape

torch.Size([64, 10])

Train and evaluate the network with forward gradient

In [20]:
def accuracy(net, loader, device):
  # A function that aggregates the accuracy over all mini-batches in the loader.
  # See here for a quick-start on torchmetrics: https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html.
  #acc = torchmetrics.Accuracy().to(device)
  acc = torchmetrics.Accuracy('multiclass', num_classes=10)
  for xb, yb in loader:
      xb, yb = xb, yb
      ypred = LG(xb)
      _ = acc(ypred, yb)
  return acc.compute()

In [21]:
# Average accuracy at initialization is 1/64% (random guessing).
accuracy(LG, test_loader, device)

tensor(0.0783)

Define Cross Entropy

In [22]:
# Note: it is important to move the CNN to the device before initializing the optimizer,
# since the optimizer also has a state that must be moved to the GPU.
loss = nn.CrossEntropyLoss()

In [23]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [24]:
def rosenbrock_function(x):
  print(x.size(0))
  sum=0
  for i in range (x.size(0) -1):
    sum += (100*torch.pow(x[i+1] - torch.pow(x[i], 2), 2) + torch.pow(x[i]-1, 2))
  return sum

In [25]:
def train_fp(x, y):
  x, y = x.to(device), y.to(device)

  l_rate0 = 0.001
  f = rosenbrock_function

  #Parameters
  w = torch.randn((64, 28*28), requires_grad=False)
  b = torch.randn(64, requires_grad=False)

  error = 1

  t=torch.tensor([0])

  while (error>1e-10) :

    t=t+1
    w1 = w.reshape(-1)
    w1 = torch.cat((w1, b), 0)
    print('w1 con b', w1.size())
    v=torch.normal(torch.zeros_like(w1),torch.ones_like(w1))

    ft = f(w1)
    print('ft', ft)
    dt=torch.tensor(torch.autograd.functional.jvp(f,w1,v)[1])
    gt=v*dt
    theta0 = theta1
    theta1 = theta1 - l_rate0*gt

  return theta1 

In [ ]:
x= torch.tensor([2, 4, 8, 16])
y=torch.tensor([1,2,3,4,5,6,7,8,9,10])
theta=train_fp(x,y)

w1 con b torch.Size([50240])
50240
ft tensor(20037906.)
50240
